In [1]:
import json
import pickle
import csv
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-05-31 23:11:35.594761: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 23:11:38.071948: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
ncol = 384
nlev = 60
tlen = 2543616

In [3]:
# Convert to pressure
def convert_to_Pressure(hyai,hybi,PS,P0):
    Dimension1=hyai.shape
    Dimension2=PS.shape
    Pressure = np.zeros([Dimension2[1],Dimension1[0]])
    
    for i in range(Dimension2[1]):
        Pressure[i,:] = (P0*hyai[:] + PS[0,i]*hybi[:])
    return Pressure

In [4]:
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')
lon = ne4_grid_info.lon.values
lat = ne4_grid_info.lat.values
area = ne4_grid_info.area.values
hyai = ne4_grid_info.hyai.values
hybi = ne4_grid_info.hybi.values
PS = ne4_grid_info.PS.values
P0 = ne4_grid_info.P0.values

In [5]:
Pre = convert_to_Pressure(hyai,hybi,PS,P0)

In [6]:
dPre = Pre[:,1:nlev+1] - Pre[:,0:nlev] 

In [7]:
mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')
ne4_grid_info = xr.open_dataset('./test_data/E3SM-MMF_ne4_grid-info.orig.nc')

def load_nc_dir_with_generator_test(filelist:list, latlim=[-999,999], lonlim=[-999,999]):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            ds = ds.merge(ne4_grid_info[['lat','lon']])
            ds = ds.where((ds['lat']>latlim[0])*(ds['lat']<latlim[1]),drop=True)
            ds = ds.where((ds['lon']>lonlim[0])*(ds['lon']<lonlim[1]),drop=True)
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            dso = dso.merge(ne4_grid_info[['lat','lon']])
            dso = dso.where((dso['lat']>latlim[0])*(dso['lat']<latlim[1]),drop=True)
            dso = dso.where((dso['lon']>lonlim[0])*(dso['lon']<lonlim[1]),drop=True)
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

In [10]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [11]:
# validation dataset for HPO
stride_sample = 5 
#f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-0[23456789]-*-*.nc')
#f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0008-1[012]-*-*.nc')
f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0009-01-*-*.nc')
#f_mli_val = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli = f_mli3
print(f'#files: {len(f_mli)}')

#files: 2232


In [12]:
tds_test = load_nc_dir_with_generator_test(f_mli)
work = list(tds_test.as_numpy_iterator())
x_true_n = np.concatenate([ work[k][0] for k in range(len(work)) ])
y_true_n = np.concatenate([ work[k][1] for k in range(len(work)) ])

2023-05-31 23:14:06.950398: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


In [13]:
# This function turns 20 minute increments into daily averages
# (384)*ndays*72 = Dim[0]
# Output dim: (384)*ndays
# Griffin's original version is based on regrided data
# This version is based on ne4pg2 grid (384 columns)
# This change allow me to use Sungduk's code directly
# since he generaged the min/max/scale already
def average_data_Griffin_LiranEdited(reconstructed_targets,reconstructed_features):
    Dim = reconstructed_targets.shape
    x = 384
    tnum = 72
    t = Dim[0]
    z = Dim[1]
    ndays = int(t/(tnum*x))
    
    target_days = np.zeros(shape=(x*ndays,tnum, z))
    feat_days = np.zeros(shape=(x*ndays,tnum, z))
    day_array_targ = np.zeros(shape=(x,tnum,ndays, z))
    day_array_feat = np.zeros(shape=(x,tnum,ndays, z))
    #print(day_array_feat.shape)
    ncol_count = 0
    tstep_count = 0
    day_count = 0
    
    for i in range(t):
        temp_targ = reconstructed_targets[i, :]
        day_array_targ[ncol_count,tstep_count,day_count, :] = temp_targ
        temp_feat = reconstructed_features[i, :]
        day_array_feat[ncol_count,tstep_count,day_count,:] = temp_feat
        
        if (ncol_count == x-1):
            ncol_count = 0
            tstep_count = tstep_count+1
        else:
            ncol_count = ncol_count+1
        
        if (tstep_count == tnum):
            tstep_count = 0
            day_count = day_count+1   
            
            
    day_array_targ_out = np.nanmean(day_array_targ, axis = 1)
    day_array_feat_out = np.nanmean(day_array_feat, axis = 1)
    
    return day_array_targ_out,day_array_feat_out

In [14]:
import os
import netCDF4 as nc
folder_path = "/pscratch/sd/h/heroplr/step2_retrain/backup_phase-4_retrained_models"  # Replace with the actual folder path
out_folder = "/pscratch/sd/h/heroplr/R2_analysis_all/"
Dim_true = x_true_n.shape
# Loop through all files in the folder
numday = int(Dim_true[0]/ncol/72)
filename="step2_lot-101_trial_0028.best.h5"
file_path = os.path.join(folder_path, filename)

In [15]:
model = keras.models.load_model(file_path,compile=False)
y_pred = model(x_true_n)
T_tend_true = y_true_n[:,:60]
T_pred_true = y_pred[:,:60]
Q_tend_true = y_true_n[:,60:120]
Q_pred_true =y_pred[:,60:120]

In [17]:
mweightpre = np.zeros([ncol,nlev])
for k in range(nlev):
    mweightpre[:,k]    =  dPre[:,k]/9.8  # dP/g

In [18]:
T_tend_true2,T_pred_true2 = average_data_Griffin_LiranEdited(T_tend_true,T_pred_true)
Q_tend_true2,Q_pred_true2 = average_data_Griffin_LiranEdited(Q_tend_true,Q_pred_true)

In [19]:
T_tend_true_daymean = np.mean(T_tend_true2,axis=1)
T_pred_true_daymean = np.mean(T_pred_true2,axis=1)
Q_tend_true_daymean = np.mean(Q_tend_true2,axis=1)
Q_pred_true_daymean = np.mean(Q_pred_true2,axis=1)

In [20]:
cp_dry_o   = 1006
latent_heat_vaporization = 2.501e6 
T_tend_true_mweight    = T_tend_true_daymean*mweightpre*cp_dry_o
T_pred_true_mweight    = T_pred_true_daymean*mweightpre*cp_dry_o
Q_tend_true_mweight    = Q_tend_true_daymean*mweightpre*latent_heat_vaporization
Q_pred_true_mweight    = Q_pred_true_daymean*mweightpre*latent_heat_vaporization 

In [21]:
# Create a new NetCDF file
filename = file_path[-31:]+"_mwv7.nc"
file_path_out = os.path.join(out_folder, filename)
        
ncfile = nc.Dataset(file_path_out, "w", format="NETCDF4")

# Define the dimensions
time_dim = ncfile.createDimension("time", None)  # Unlimited dimension
lat_dim = ncfile.createDimension("ncol", ncol)
lon_dim = ncfile.createDimension("nlev", nlev)
day_dim = ncfile.createDimension("nday", numday)
# Create variables
time_var = ncfile.createVariable("time", "f8", ("time",))
lon_var = ncfile.createVariable("lon", "f4", ("ncol",))
lat_var = ncfile.createVariable("lat", "f4", ("ncol",))
data_var3 = ncfile.createVariable("mwT_tend", "f8", ("nlev","ncol"))
data_var4 = ncfile.createVariable("mwT_pred", "f8", ("nlev","ncol"))
data_var5 = ncfile.createVariable("mwQ_tend", "f8", ("nlev","ncol"))
data_var6 = ncfile.createVariable("mwQ_pred", "f8", ("nlev","ncol"))

# Assign values to variables
time_var[:] = [1]  # Example time values
lon_var[:] = lon    # Example latitude values
lat_var[:] = lat  # Example longitude values

data_var3[:,:] = np.transpose(T_tend_true_mweight)
data_var4[:,:] = np.transpose(T_pred_true_mweight)
data_var5[:,:] = np.transpose(Q_tend_true_mweight)
data_var6[:,:] = np.transpose(Q_pred_true_mweight)
# Add global attributes
ncfile.description = "R2"
ncfile.history = "Created by Liran"

# Close the NetCDF file
ncfile.close()
    

In [37]:
Q_pred_true_daymean*mweightpre*latent_heat_vaporization

array([[ 7.91704547e-01,  1.97721543e+01,  9.14825877e-03, ...,
        -2.11405090e+07, -1.90902322e+07, -1.53731224e+07],
       [ 7.98493601e-01,  2.04567410e+01,  4.82251249e-01, ...,
        -2.46353390e+07, -2.63060303e+07, -2.22593401e+07],
       [ 8.41622522e-01,  2.29387579e+01,  1.36609745e-01, ...,
        -3.32867803e+07, -2.94096615e+07, -2.39751447e+07],
       ...,
       [ 2.69922849e-01,  1.02774629e+01,  9.42671631e-01, ...,
        -1.70180005e+06,  3.44317063e+06,  1.43468133e+07],
       [ 5.70988483e-01,  1.70196957e+01, -4.37779091e-02, ...,
         1.69786125e+07,  2.64174654e+07,  2.96745203e+07],
       [ 5.91387573e-01,  1.57626410e+01,  4.15191256e-01, ...,
         8.47490755e+06,  1.56238896e+07,  1.68826972e+07]])

In [39]:
Q_tend_true_daymean*mweightpre

array([[  0.        ,   0.        ,   0.        , ...,  -9.29880766,
         -7.8545083 ,  -7.57065786],
       [  0.        ,   0.        ,   0.        , ..., -10.6481393 ,
        -11.45301205, -10.43584406],
       [  0.        ,   0.        ,   0.        , ..., -11.11197566,
        -10.35175524,  -9.53319232],
       ...,
       [  0.        ,   0.        ,   0.        , ...,  -1.53546201,
          0.73138866,   5.0305411 ],
       [  0.        ,   0.        ,   0.        , ...,   7.18744831,
          9.08036054,  10.00338113],
       [  0.        ,   0.        ,   0.        , ...,   2.63099588,
          4.84712823,   5.87937678]])

In [35]:
T_tend_true_daymean*mweightpre*cp_dry_o

array([[ 6.26984715e+00,  4.27084536e+00,  3.08678181e+01, ...,
        -8.71830039e+01, -1.17413186e+03, -4.52326301e+03],
       [ 7.00858172e+00,  2.94994946e+00,  3.15487315e+01, ...,
         8.09643166e+02, -2.54908567e+02, -4.12383654e+03],
       [ 4.25685480e+00,  3.51064717e-01,  1.83241209e+01, ...,
         6.14965720e+02, -1.97795536e+03, -1.00127329e+04],
       ...,
       [-1.01284031e+00, -2.12884899e+01, -3.04141918e+01, ...,
        -2.13660916e+03, -4.96020327e+03, -9.70513408e+03],
       [-1.56064617e+00, -2.53805508e+01, -3.58115412e+01, ...,
        -6.30918793e+03, -1.22065922e+04, -2.23815776e+04],
       [-1.54803969e+00, -2.89294434e+01, -3.42962165e+01, ...,
        -5.19248918e+03, -1.05121594e+04, -1.99591239e+04]])

In [36]:
T_pred_true_daymean*mweightpre*cp_dry_o

array([[ 6.01962860e+00,  7.38933522e+00,  3.76484774e+01, ...,
        -4.02879291e+02, -1.57335896e+03, -5.28923856e+03],
       [ 7.18102170e+00,  4.15699208e+00,  3.68084969e+01, ...,
         4.50867126e+02, -5.98607804e+02, -4.74713790e+03],
       [ 3.31197719e+00,  3.46639950e+00,  2.46805505e+01, ...,
         5.88821959e+02, -2.54821723e+03, -1.07308687e+04],
       ...,
       [ 4.72626600e-01, -2.14670692e+01, -2.81643660e+01, ...,
        -2.33192802e+03, -5.16943392e+03, -1.01330786e+04],
       [-2.25709572e+00, -2.73062648e+01, -3.07144266e+01, ...,
        -6.36864550e+03, -1.24304647e+04, -2.20538137e+04],
       [-2.61002373e+00, -2.93857447e+01, -2.91604102e+01, ...,
        -5.65467515e+03, -1.09535505e+04, -2.03086027e+04]])

In [42]:
Pre[3,:]

array([5.58810705e+00, 1.00814552e+01, 1.81402085e+01, 3.24444509e+01,
       5.74056761e+01, 9.98635562e+01, 1.69607596e+02, 2.79347861e+02,
       4.43938435e+02, 6.79228850e+02, 1.00142179e+03, 1.42747608e+03,
       1.97588953e+03, 2.66627009e+03, 3.51659916e+03, 4.53891697e+03,
       5.73600949e+03, 7.10183619e+03, 8.62609533e+03, 1.02999231e+04,
       1.21183316e+04, 1.40772291e+04, 1.61670345e+04, 1.83692322e+04,
       2.06759858e+04, 2.30328226e+04, 2.54156347e+04, 2.78156725e+04,
       3.02400640e+04, 3.27059992e+04, 3.52325139e+04, 3.78342192e+04,
       4.05191007e+04, 4.32895607e+04, 4.61440398e+04, 4.90766978e+04,
       5.20744951e+04, 5.51134202e+04, 5.81570611e+04, 6.11601554e+04,
       6.40771765e+04, 6.68727465e+04, 6.95288798e+04, 7.20452832e+04,
       7.44326781e+04, 7.67029766e+04, 7.88616250e+04, 8.09057959e+04,
       8.28285929e+04, 8.46262341e+04, 8.63040154e+04, 8.78781103e+04,
       8.93728764e+04, 9.08155985e+04, 9.22313283e+04, 9.36396804e+04,
      